# Step -1 : Business  Problem Understanding

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Step -2 : Data Understaning

Load Data & Understand Every variable

In [3]:
df = pd.read_excel("insurance.xlsx")
df.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


# Dataset Understanding

In [4]:
df.shape

(1338, 7)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


# Exploratory data analysis

In [8]:
Categorical=[]
continous=[]
check=[]

d_types= dict(df.dtypes)

for name, type in d_types.items():
    if str(type)=='object':
        Categorical.append(name)
        
    elif str(type)=='float64':
        continous.append(name)
    
    else:
        
        check.append(name)

print("categorical:",Categorical)
print("Continous",continous)
print("Check",check)

categorical: ['sex', 'smoker', 'region']
Continous ['bmi', 'expenses']
Check ['age', 'children']


In [13]:
d_types = dict(df.dtypes)

for name, type_  in  d_types.items():
    if str(type_)== 'object':
        print(f"======{name}====>")
        print(df[name].value_counts())

======sex====>
male      676
female    662
Name: sex, dtype: int64
======smoker====>
no     1064
yes     274
Name: smoker, dtype: int64
======region====>
southeast    364
southwest    325
northwest    325
northeast    324
Name: region, dtype: int64


In [14]:
df.describe()

,age,bmi,children,expenses
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.665471,1.094918,13270.422414
std,14.049960,6.098382,1.205493,12110.011240
min,18.000000,16.000000,0.000000,1121.870000
25%,27.000000,26.300000,0.000000,4740.287500
50%,39.000000,30.400000,1.000000,9382.030000
75%,51.000000,34.700000,2.000000,16639.915000
max,64.000000,53.100000,5.000000,63770.430000


In [15]:
df.corr()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15280\1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,age,bmi,children,expenses
age,1.000000,0.109341,0.042469,0.299008
bmi,0.109341,1.000000,0.012645,0.198576
children,0.042469,0.012645,1.000000,0.067998
expenses,0.299008,0.198576,0.067998,1.000000


# step -3 : Data Processing

In [16]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
expenses    0
dtype: int64

In [17]:
#drop the region columns (unwanted column)

df.drop('region',axis=1,inplace=True)


In [26]:
# Encoding sex column
df['sex'].replace({'female':0,'male':1},inplace=True)


#Encoding smoker columns

df['smoker'].replace({'no':0,'yes':1},inplace=True)



# X & Y

In [27]:
x=df.drop(columns='expenses')
y=df['expenses']

In [41]:
# Train test split

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=20,random_state=37)

# Step 4,5 : Modelling and Evaluation

In [46]:
#Modelling 

from sklearn.linear_model import Ridge

ridge_model=Ridge()

ridge_model.fit(x_train,y_train)

#Prediction

train_predictions=ridge_model.predict(x_train)
test_predictions=ridge_model.predict(x_test)

#R2

print("train_R2",ridge_model.score(x_train,y_train))

print("test_R2",ridge_model.score(x_test,y_test))


#Cross validation score

from sklearn.model_selection import cross_val_score
cv=cross_val_score(ridge_model,x,y,cv=5).mean()
cv


train_R2 0.7492043591292339
test_R2 0.790993519548942


0.7466523551462286

# Applying Hyper Parameter tunning for Ridge Regression


In [48]:
from sklearn.model_selection import GridSearchCV

#model

estimator=Ridge()

#Parameters and values

param_grid = {"alpha":[0.1,0.2,0.5,0.71,10,100,1000]}

#Identifying the best value of the parameter within given values for the given data

model_hp = GridSearchCV(estimator,param_grid,cv=5,scoring='neg_mean_squared_error')

model_hp.fit(x_train,y_train)

model_hp.best_params_

{'alpha': 0.1}

# Rebuild Ridge model using best hyperparameter

In [52]:
#modelling 

ridge_best = Ridge(alpha=0.1)
ridge_best.fit(x_train,y_train)

print("Intercept",ridge_best.intercept_)
print("Coefficient",ridge_best.coef_)

#prediction
train_predictions = ridge_best.predict(x_train)
test_predictions=ridge_best.predict(x_test)

#Evaluation

print("Train R2:",ridge_best.score(x_train,y_train))
print("test R2:",ridge_best.score(x_test,y_test))
print("cross validation Score",cross_val_score(ridge_best,x,y,cv=5).mean())

Intercept -12117.953355964588
Coefficient [  259.60182408  -165.30419369   323.09833309   479.99045792
 23786.1456619 ]
Train R2: 0.7492177677423766
test R2: 0.7914846409748292
cross validation Score 0.7466599257222318


# Prediction on new data

In [53]:
input_data={"age":31,
           "sex":"female",
           "bmi":25.74,
           "children":0,
           'smoker':"no",
           "region":"northest"}

In [60]:
#preprocessing data
df_test=pd.DataFrame(input_data,index=[0])
df_test.drop(columns='region',inplace=True)
df_test['sex'].replace({'female':0,'male':1},inplace=True)
df_test['smoker'].replace({'no':0,'yes':1},inplace=True)

transformed_data=df_test

# Predict

In [61]:
ridge_best.predict(transformed_data)

array([4246.25428408])